In [1]:
# conda env: pyg (Python3.9.16)
# --> 
# 1. improve it by adding multiprocessing feature
# 2. add scaffold split for classification task. Because now three spliting methods:
#    2.1 cluster split + stratified by cliff

import os
import sys
from typing import List, Union
import shutil

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.cluster import SpectralClustering

from datacat4ml.utils import mkdirs, get_df_name
from datacat4ml.const import *
from datacat4ml.Scripts.data_prep.data_split.split_utils.cliff import ActivityCliffs, get_tanimoto_matrix

# Split the categorized datasets

In [2]:
def data_spliter(smiles: List[str], pStandard_value: Union[List[float], np.array], activity: List[float],
                 similarity: float=0.9, potency_fold: int=1, 
                 use_clustering: bool=True, n_clusters: int=5, test_size: float=0.2, task: str='cls') -> pd.DataFrame:
    """
    Split the data into train and test sets according to activity cliffs and compound charateristics.

    :param smiles: List of SMILES strings
    :param pStandard_value: List of pStandard_values or np.array of pStandard_values
    :param activity: List of active/inactive labels, where 1 is active and 0 is inactive
    :param similarity: Threshold value to determine structural similarity
    :param potency_fold: Threshold value to determine potency difference, where the potency here is the pStandard_value
    :param use_clustering: Whether to use clustering to split the data
    :param n_clusters: Number of clusters to use if clustering is used
    :param test_size: Test set size

    :return: A dataframe 
    """

    if len(smiles) < 50:
        print(f"The number of data points in this file is lower than 50, which is not enough to build a good machine learning model. Skipping this file.")
        return None

    if use_clustering:
        # # cluster the dabaset into 5 clusters based on tanimoto distance matrix
        spectral = SpectralClustering(n_clusters=n_clusters, affinity='precomputed', random_state=RANDOM_SEED)
        clusters = spectral.fit(get_tanimoto_matrix(smiles)).labels_ # get the cluster labels for each molecule

        train_idx, test_idx = [], []
        for cluster in range(n_clusters):
            num_in_cluster = len(np.where(clusters == cluster)[0])
            print(f"Cluster {cluster}: {num_in_cluster} data points")

            # get the indices of molecules in the current cluster
            cluster_idx = np.where(clusters == cluster)[0] # `[0]` is need to convert the tuple to a list

            if task == 'reg':
                # get activity cliffs
                cliffs = ActivityCliffs(smiles, pStandard_value)
                cliff_mols = cliffs.get_cliff_molecules(return_smiles=False, similarity=similarity, potency_fold=potency_fold)
                clust_cliff_mols = [cliff_mols[i] for i in cluster_idx]
            
                # can only split data stratified by cliff molecules if there are more than 1 cliff molecules in the cluster, else just split randomly
                if sum(clust_cliff_mols) > 2:
                    try: # try to avoid the Value ERROR when the number of class in the training set is less than 2
                        clust_train_idx, clust_test_idx = train_test_split(cluster_idx, test_size=test_size, 
                                                                            stratify=clust_cliff_mols, # ensure that the train/test split has the same proportion of cliff molecules
                                                                            random_state=RANDOM_SEED, shuffle=True)
                    except ValueError:
                        # This block will be executed if an error occurs in the try block
                        print("An error occurred while stratifying data based on cliff molecules. Skipping stratification.")
                        clust_train_idx, clust_test_idx = train_test_split(cluster_idx, test_size=test_size, 
                                                                            random_state=RANDOM_SEED, shuffle=True)
                else:
                    clust_train_idx, clust_test_idx = train_test_split(cluster_idx, test_size=test_size, 
                                                                        random_state=RANDOM_SEED, shuffle=True)

            elif task == 'cls':
                print("Classification task")
                clust_train_idx, clust_test_idx = train_test_split(cluster_idx, test_size=test_size, 
                                                                    random_state=RANDOM_SEED, shuffle=True)
                print(f'done splitting in task {task}')

            train_idx.extend(clust_train_idx)
            test_idx.extend(clust_test_idx)
            
    else:
        # don't use clustering before spliting. This is to avoid potential cheating by using clustering to split the data, 
        # which make the structual diversity of the train and test sets the same
        if task == 'reg':
            if sum(cliff_mols) > 2:
                train_idx, test_idx = train_test_split(range(len(smiles)), test_size=test_size, 
                                                stratify=cliff_mols, # ensure that the train/test split has the same proportion of cliff molecules
                                                random_state=RANDOM_SEED, shuffle=True)
            else:
                train_idx, test_idx = train_test_split(range(len(smiles)), test_size=test_size, 
                                                random_state=RANDOM_SEED, shuffle=True)
        elif task == 'cls':
            train_idx, test_idx = train_test_split(range(len(smiles)), test_size=test_size, 
                                                random_state=RANDOM_SEED, shuffle=True)
            
    train_test = []
    for i in range(len(smiles)):
        if i in train_idx:
            train_test.append('train')
        elif i in test_idx:
            train_test.append('test')
        else:
            raise ValueError('Index not in train or test set')
    
    active_ratio = sum(activity) / len(activity)
    active_ratio_train = sum([activity[i] for i in train_idx]) / len(train_idx)
    active_ratio_test = sum([activity[i] for i in test_idx]) / len(test_idx)
    
    if task == 'reg':
        return pd.DataFrame({'active_ratio': active_ratio,
                            'active_ratio_train': active_ratio_train,
                            'active_ratio_test': active_ratio_test,
                            'cliff_mol': cliff_mols,
                            'split': train_test})
    elif task == 'cls':
        return pd.DataFrame({'active_ratio': active_ratio,
                            'active_ratio_train': active_ratio_train,
                            'active_ratio_test': active_ratio_test,
                            'split': train_test})

In [3]:
def split_data(input_filepath = CURA_CAT_DATASETS_DIR, output_filepath = SPLIT_CAT_DATASETS_DIR,
               task:str = 'cls', use_clustering: bool=True):
    
    # access the final csv files obtained from data curation
    folder_path = os.path.join(input_filepath, task)
    files = os.listdir(folder_path)
    curated_files = [file for file in files if file.endswith('curated.csv')]

    # make new directory to store the featurized data
    output_path = os.path.join(output_filepath, task, 'use_clustering'+'_'+str(use_clustering))
    if os.path.exists(output_path):
        shutil.rmtree(output_path)
    mkdirs(output_path)

    for curated_file in curated_files:
        print (f"curated_file is: {curated_file}\n")

        df = pd.read_csv(os.path.join(folder_path, curated_file))
        df = df.drop(columns=['Unnamed: 0'])

        # split the data into train and test sets
        data_splited_df = data_spliter(df['canonical_smiles_by_Std'].tolist(), df['pStandard_value'].tolist(), df['activity'].tolist(), 
                                       use_clustering=use_clustering, task=task)
        
        # save data_splited_df as a csv file if it is not None
        if data_splited_df is not None:
            concat_df = pd.concat([df, data_splited_df], axis=1)
            concat_df.to_csv(os.path.join(output_path, f'{curated_file[:-11]}split.csv'), index=False)

In [ ]:
use_clusterings = [True, False]

for task in Tasks:
    print (f"task is: {task}\n")
    
    for use_clustering in use_clusterings:
        print (f"use_clustering is: {use_clustering}\n")
        
        split_data(input_filepath = CURA_CAT_DATASETS_DIR, output_filepath = SPLIT_CAT_DATASETS_DIR, 
                   task=task, use_clustering=use_clustering)
        print(f'Done!\n====================================\n')
#It took around 17 mins to run the above code locally. 
# It's okay to encounter error for 'reg + use_clustering=False' because no calulcation for identifying cliff mols without using clustering.

task is: cls

use_clustering is: True

curated_file is: nor_antag_G_GTP_IC50_curated.csv



100%|██████████| 150/150 [00:00<00:00, 7141.67it/s]


Cluster 0: 32 data points
Classification task
done splitting in task cls
Cluster 1: 26 data points
Classification task
done splitting in task cls
Cluster 2: 44 data points
Classification task
done splitting in task cls
Cluster 3: 40 data points
Classification task
done splitting in task cls
Cluster 4: 8 data points
Classification task
done splitting in task cls
curated_file is: kor_antag_B_arrest_IC50_curated.csv



100%|██████████| 53/53 [00:00<00:00, 13094.08it/s]

Cluster 0: 18 data points
Classification task
done splitting in task cls
Cluster 1: 12 data points
Classification task
done splitting in task cls
Cluster 2: 6 data points
Classification task
done splitting in task cls
Cluster 3: 7 data points
Classification task
done splitting in task cls
Cluster 4: 10 data points
Classification task
done splitting in task cls
curated_file is: nor_agon_G_Ca_EC50_curated.csv

The number of data points in this file is lower than 50, which is not enough to build a good machine learning model. Skipping this file.
curated_file is: dor_antag_B_arrest_IC50_curated.csv

The number of data points in this file is lower than 50, which is not enough to build a good machine learning model. Skipping this file.
curated_file is: dor_agon_G_cAMP_IC50_curated.csv

The number of data points in this file is lower than 50, which is not enough to build a good machine learning model. Skipping this file.
curated_file is: kor_agon_G_cAMP_IC50_curated.csv

The number of data po

...

100%|██████████| 672/672 [00:00<00:00, 5973.89it/s]


Cluster 4: 62 data points


100%|██████████| 672/672 [00:00<00:00, 5938.12it/s]


Done!

use_clustering is: False

curated_file is: nor_antag_G_GTP_IC50_curated.csv



UnboundLocalError: local variable 'cliff_mols' referenced before assignment